# 误差反向传播法

计算图（Computation Graph） 是深度学习中的一个重要概念，用于表示计算过程。它是一个有向无环图（Directed Acyclic Graph, DAG），其中每个节点表示一个操作或变量，每条边表示操作之间的依赖关系。

<span style="color: red;">**计算图的组成**</span>
1. 节点（Nodes）：
* 变量节点：表示输入数据、权重、偏置等变量。
* 操作节点：表示数学运算（如加法、乘法、激活函数等）。

2. 边（Edges）：
* 表示数据在节点之间的流动和依赖关系。